# Japanese emotion analysis
Based on SNOW D18: Japanese Emotional Expression Dictionary from http://www.jnlp.org/SNOW/D18

Further corpuses available: https://lionbridge.ai/datasets/japanese-language-text-datasets/

## Prerequisites
kytea required to tokenize Japanese text. Install using
```
wget http://www.phontron.com/kytea/download/kytea-0.4.7.tar.gz
tar -xvf kytea-0.4.7.tar
cd kytea-0.4.7
./configure
make
make install
```

## Prepare tokenizer
Use Mykytea tokenizer to split text. Prepare also a set of kanas to avoid assigning emotions to kanas.

In [264]:
import Mykytea

In [265]:
opt = "-model /usr/local/share/kytea/model.bin"
mk = Mykytea.Mykytea(opt)

In [266]:
# hiragana, katakana and unionedkanas
hira = {chr(l) for l in range(12353,12439)}
kata = {chr(l) for l in range(12449,12539)}
kana = hira.union(kata)

## Prepare emotion corpus
Load emotion corpus with expressions and multiple assigned emotions per expression. 
Prepare tokenized and "stemmed" dictionary for emotions lookup in next step.

In [205]:
import pandas as pd

In [208]:
ecorp = pd.read_csv('data/D18-2018.7.24.csv')
print(ecorp)
available_emotions = pd.read_csv('data/emotion_assignment.csv')
print(available_emotions)

       Word Katakana Emotion
0         哀       アイ       悲
1         愛       アイ       好
2     相いれない   アイイレナイ       嫌
3        哀感     アイカン       悲
4        愛敬    アイキョウ      好親
...     ...      ...     ...
1973    悪賢い   ワルガシコイ       嫌
1974   悪びれる    ワルビレル       謝
1975  我を忘れる  ワレオワスレル       興
1976     わぁ       ワー       喜
1977    わーい      ワーイ       喜

[1978 rows x 3 columns]
        Emotion Symbol
0           安らぎ      安
1           楽しさ      楽
2           親しみ      親
3         尊敬・尊さ      尊
4            感謝      感
5        気持ちが良い      気
6          誇らしい      誇
7            感動      動
8            喜び      喜
9           悲しさ      悲
10          寂しさ      寂
11           不満      不
12          切なさ      切
13          苦しさ      苦
14           不安      ふ
15           憂鬱      鬱
16           辛さ      辛
17           好き      好
18           嫌悪      嫌
19        恥ずかしい      恥
20           焦り      焦
21           驚き      驚
22           怒り      怒
23          幸福感      幸
24           恨み      恨
25  恐れ（恐縮等の意味で）      恐
26    

Prepare dictionary to map short form of emotion to more understandable longer form of emtion: 悲 -> 悲しさ or 恐 -> 恐れ（恐縮等の意味で）

In [209]:
emotion_map = {row['Symbol']: row['Emotion'] for index, row in available_emotions.iterrows()}
emotion_map.update({'面':''})
emotion_map.items()

dict_items([('安', '安らぎ'), ('楽', '楽しさ'), ('親', '親しみ'), ('尊', '尊敬・尊さ'), ('感', '感謝'), ('気', '気持ちが良い'), ('誇', '誇らしい'), ('動', '感動'), ('喜', '喜び'), ('悲', '悲しさ'), ('寂', '寂しさ'), ('不', '不満'), ('切', '切なさ'), ('苦', '苦しさ'), ('ふ', '不安'), ('鬱', '憂鬱'), ('辛', '辛さ'), ('好', '好き'), ('嫌', '嫌悪'), ('恥', '恥ずかしい'), ('焦', '焦り'), ('驚', '驚き'), ('怒', '怒り'), ('幸', '幸福感'), ('恨', '恨み'), ('恐', '恐れ（恐縮等の意味で）'), ('怖', '恐怖'), ('悔', '悔しさ'), ('祝', '祝う気持ち'), ('困', '困惑'), ('き', 'きまずさ'), ('興', '興奮'), ('悩', '悩み'), ('願', '願望'), ('失', '失望'), ('憐', 'あわれみ'), ('見', '見下し'), ('謝', '謝罪'), ('た', 'ためらい'), ('Ｆ', '不快'), ('怠', '怠さ'), ('あ', 'あきれ'), ('心', '心配'), ('緊', '緊張'), ('妬', '妬み'), ('憎', '憎い'), ('残', '残念'), ('情', '情けない'), ('穏', '穏やか'), ('面', '')])

One expression may have multiple emotions assigned. Prepare map of all expressions with mapped emotions.

In [170]:
elookup_raw = {row['Word']: [emotion_map[emotion] 
                         for emotion in list(row['Emotion'])] for index, row in ecorp.iterrows()}

## Tokenize and "stemming"
Tokenize/"Stem" each expression and duplicate emotions to both parts. Do not assign emotions to tokens that only consist of one kana.

In [210]:
elookup = dict()
for exp, emotion in elookup_raw.items():
    for word in mk.getWS(exp):
        if word not in kana:
            elookup[word] = emotion

In [214]:
print(elookup_raw['飽き足りる'])
print(elookup['飽き足り'])

['不満']
['不満']


## Test corpus

In [215]:
s = 'あなた毛布取らないで'
for word in mk.getWS(s):
    print(word)

あなた
毛布
取
ら
な
い
で


In [261]:
# emotion lookup
def emotions(s):
    emotions = set()
    for word in mk.getWS(s):
        if word in elookup:
            emotions.update(elookup[word])
    return emotions

In [262]:
sentences = ['あなた毛布取らないで', 'あなたもう太らないで', '友達が寂しい',
            'おお味噌か?','ひと月居てもいい？','音した？','カナリヤ少ない','バナナを食べましたが美味しかった',
            '文句を言ったら怒られた']
[(sentence, emotions(sentence)) for sentence in sentences]

[('あなた毛布取らないで', {'あきれ', '困惑'}),
 ('あなたもう太らないで', set()),
 ('友達が寂しい', {'寂しさ'}),
 ('おお味噌か?', {'驚き'}),
 ('ひと月居てもいい？', set()),
 ('音した？', set()),
 ('カナリヤ少ない', set()),
 ('バナナを食べましたが美味しかった', {'喜び', '安らぎ', '穏やか'}),
 ('文句を言ったら怒られた', {'不満', '怒り'})]